<a href="https://colab.research.google.com/github/Super-rookie-Py/_WebProgramming_Class/blob/master/%EB%B0%95%EA%B1%B4%EC%9A%B0__%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%89%B4%EC%8A%A4_%EB%8C%93%EA%B8%80_%EC%8A%A4%ED%81%AC%EB%9E%98%ED%95%91.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selenium 네이버 뉴스 댓글 스크래핑

## Selenium 및 웹 드라이버 설치

In [1]:
!pip install Selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

     |████████████████████████████████| 911kB 2.8MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:11 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [901 kB]
Get:12 http://a

## 라이브러리 import

In [4]:
import time
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

## `news_scraping()`: 뉴스 스크래핑

In [45]:
def news_scraping(news_url, wd):

  press = wd.find_element_by_xpath('//*[@id="main_content"]/div[1]/div[1]/a/img').get_attribute('title')
  title = wd.find_element_by_id('articleTitle').text
  datetime = wd.find_element_by_class_name('t11').text
  article = wd.find_element_by_id('articleBodyContents').text

  article = article.replace("// flash 오류를 우회하기 위한 함수 추가", "")
  article = article.replace("function _flash_removeCallback() {}", "")
  article = article.replace("\n", "")
  article = article.replace("\t", "")

  good = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[1]/a/span[2]').text
  warm = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[2]/a/span[2]').text
  sad = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[3]/a/span[2]').text
  angry = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[4]/a/span[2]').text
  want = wd.find_element_by_xpath('//*[@id="spiLayer"]/div[1]/ul/li[5]/a/span[2]').text
  recom = wd.find_element_by_xpath('//*[@id="toMainContainer"]/a/em[2]').text
  
  print("뉴스:", [title, press, datetime, article, good, warm, sad, angry, want, recom, news_url])

  return [title, press, datetime, article, good, warm, sad, angry, want, recom, news_url]


## `comments_scraping()`: 뉴스 댓글 스크래핑

In [46]:
def comments_scraping(news_url, wd):

  try:
    wd.find_element_by_class_name('u_cbox_in_view_comment').click()
    print("[댓글 더보기]", end='')
    time.sleep(1)

    while True:
      wd.find_element_by_class_name('u_cbox_page_more').click()
      print("[더보기]", end = "")
      time.sleep(1)
  except:
    pass

  btn_reply_list = wd.find_elements_by_class_name('u_cbox_btn_reply')
  for btn_reply in btn_reply_list:
    btn_reply.send_keys('\n')
    print("[답글]", end = "")
    time.sleep(1)

  print("[댓글 스크래핑]")
  comments_idx = 0
  comments_df = pd.DataFrame(columns = ("Contents", "Name", "Datetime", "Recommend", "Unrecommend", "URL"))

  comments = wd.find_elements_by_class_name("u_cbox_comment_box")
  for comment in comments:
    try:
      name = comment.find_element_by_class_name('u_cbox_nick').text
      date = comment.find_element_by_class_name('u_cbox_date').text
      contents = comment.find_element_by_class_name('u_cbox_contents').text
      recomm = comment.find_element_by_class_name('u_cbox_cnt_recomm').text
      unrecomm = comment.find_element_by_class_name('u_cbox_cnt_unrecomm').text
      print(f"   댓글 #{comments_idx+1}:", [contents, name, date, recomm, unrecomm, news_url])
      comments_df.loc[comments_idx] = [contents, name, date, recomm, unrecomm, news_url]
      comments_idx += 1
    except NoSuchElementException:
      print("[삭제되거나 부적절한 댓글]")
      continue

  return comments_df  

## `scraping()`: 스크래핑 함수

In [47]:
def scraping(news_max = 3):
  wd = webdriver.Chrome('chromedriver', options=chrome_options)
  wd.implicitly_wait(3)

  wd.execute_script('window.open("about:blank", "blank");')
  tabs = wd.window_handles

  wd.switch_to.window(tabs[0])
  query = input("검색어 입력: ")
  search_url = "https://search.naver.com/search.naver?&where=news&ie=utf8&sm=nws_hty&query=" + query 
  wd.get(search_url)

  news_idx = 0
  news_df = pd.DataFrame(columns = ("Title", "Press", "Datetime", "Article", "Good","Warm","Sad","Angry","Want","Recomend", "URL"))
  comments_df = pd.DataFrame()

  while True:
    inline_list = wd.find_elements_by_class_name('txt_inline')
    for inline in inline_list:
      try:    
        news_url = inline.find_element_by_class_name('_sp_each_url').get_attribute('href')
      except:
        continue
      
      wd.switch_to.window(tabs[1])
      wd.get(news_url)
      
      news_df.loc[news_idx] = news_scraping(news_url, wd)
      news_idx += 1

      df = comments_scraping(news_url, wd)
      comments_df = pd.concat([comments_df, df])

      if news_idx >= news_max:
        break
    if news_idx >= news_max:
        break

    try:
      wd.switch_to.window(tabs[0])
      wd.find_element_by_class_name('next').click()
      time.sleep(1)
    except:
      break

  wd.close()

  return news_df , comments_df

## 스크래핑 실행

In [54]:
news_df, comments_df = scraping()

검색어 입력: 냉장고
뉴스: ['카센터로 "냉장고 배송 언제?"…쿠팡 계정 훔친 사기꾼', 'SBS', '2020.09.16. 오전 8:07', "동영상 뉴스<앵커>사기꾼 한 명이 남의 쿠팡 계정을 훔쳐 가짜 판매 글을 올리고 돈만 받아 잠적하는 일이 속출하고 있습니다. 상황이 이런 데도 쿠팡은 규정에서 벗어난 직거래였다면서 아무런 책임을 질 수 없다는 입장입니다.UBC 김예은 기자가 취재했습니다.<기자>한 카센터 운영업자는 최근 알 수 없는 냉장고 배송 문의 전화를 수백 통 받았습니다.차량 부품을 팔기 위해 쿠팡에 판매자로 등록했는데, 누군가 자신의 아이디로 냉장고 판매 글을 올린 겁니다.이들은 도용한 아이디로 저렴한 값에 물품을 배송한다며 구매자들을 속인 뒤, 자신들의 계좌번호로 입금하게 하는 방식으로 수천만 원을 챙겼습니다.[권 모 씨/계정 도용 피해자 : (쿠팡에) 피해자가 많을 수 있다, 전화를 주고 문자를 주든지 해서 구매 취소를 하고 상품을 내려 달라고 했는데, 오늘까지도 (구매자에게) 전화 두 통 또 받았어요.]피해자들이 구매한 페이지에 들어가 보니 여전히 구매가 가능한데요, 특가 제품이라며 구매 전 연락을 유도하고 있습니다.배송되지도 않는 가전제품 주문으로 피해를 본 사람은 확인된 사람만 40여 명, 300만 원을 입금한 한 피해자는 문의 직후 쿠팡고객센터라는 가짜 문자 메시지로 개별 계좌를 알려줘 사기를 의심하지 않았다고 말합니다.[구매 사기 피해자 : '쿠팡을 해킹해?' 이런 건 저희가 상상할 수 없었어요. 그러니까 의심을 할 수 없죠.]하지만 마켓 측은 지정된 계좌가 아닌 개별 계좌를 통한 구매 피해라 책임질 수 없다는 입장입니다.다른 오픈마켓에서도 최근 판매자의 아이디를 도용한 사기 피해가 잇따르면서 피해자들은 국민청원을 올리고 경찰에 수사를 의뢰했습니다.   UBC 김예은▶ [마부작침] 2020 또 다른 '조두순'들은 지금▶ [뉴스속보] 코로나19 재확산 현황▶ 더 깊은 인물 이야기 '그, 사람'※ ⓒ SBS & SBS Digital 

In [55]:
news_df

,Title,Press,Datetime,Article,Good,Warm,Sad,Angry,Want,Recomend,URL
0,"카센터로 ""냉장고 배송 언제?""…쿠팡 계정 훔친 사기꾼",SBS,2020.09.16. 오전 8:07,동영상 뉴스<앵커>사기꾼 한 명이 남의 쿠팡 계정을 훔쳐 가짜 판매 글을 올리고 돈...,2,1,1,72,5,18,https://news.naver.com/main/read.nhn?mode=LSD&...
1,사라진 냉장고 안 맥주캔… 범인은 윗집 남자였다,국민일보,2020.09.10. 오전 9:53,나체로 음란행위까지… CCTV 확인한 집주인이 신고게티이미지뱅크부산에서 20대 남성...,5,3,1,485,9,82,https://news.naver.com/main/read.nhn?mode=LSD&...
2,냉장고 맥주 사라져 CCTV 봤더니…윗집 남자가 알몸으로,동아일보,2020.09.10. 오전 8:28,"부산에서 원룸 복도를 알몸으로 돌아다니며 음란행위를 하고, 아랫층 거주자의 집에 몰...",1,0,0,21,0,4,https://news.naver.com/main/read.nhn?mode=LSD&...


In [61]:
comments_df

,Contents,Name,Datetime,Recommend,Unrecommend,URL
0,안타까운 일이지만 3백이나 넣은 40명 분들 생각 좀 해봐요.. 다 떠나서 쿠팡 계...,heav****,2020.09.15. 23:27,172,10,https://news.naver.com/main/read.nhn?mode=LSD&...
1,쿠팡은 시스템에서 같은 모델의 제품이라고 인식하면 상품페이지는 물론 리뷰까지 통합해...,aosd****,2020.09.16. 01:13,2,1,https://news.naver.com/main/read.nhn?mode=LSD&...
2,"또한 꼼꼼히 알아보지 못한 소비자의 책임도 있지만, 과거부터 최근까지 비슷한 사례의...",aosd****,2020.09.16. 01:19,2,0,https://news.naver.com/main/read.nhn?mode=LSD&...
3,저기 나와있는 사람들은 조금 더 싸게 사려고 판매자와 직거래인점 확실히 인지하고 입...,rhtn****,2020.09.16. 02:00,2,1,https://news.naver.com/main/read.nhn?mode=LSD&...
4,결제 페이지에서 입금하고 당한거라면 두말 할것도 없이 전적으로 쇼핑몰에서 책임져야되...,perf****,2020.09.16. 02:11,2,0,https://news.naver.com/main/read.nhn?mode=LSD&...
...,...,...,...,...,...,...
8,정치 병자들 치료 받드세요 가족들 힘들게 하지말고,okaj****,2020.09.09. 23:50,1,1,https://news.naver.com/main/read.nhn?mode=LSD&...
9,더블당 청년인재 영입1호,sr71****,2020.09.09. 23:50,25,2,https://news.naver.com/main/read.nhn?mode=LSD&...
10,경상도 아니면 이런기사는 볼수가없네~~,hoob****,2020.09.09. 23:48,1,8,https://news.naver.com/main/read.nhn?mode=LSD&...
11,윤미향 추미애 조국 같은 넘일세 ㅎㅎㅎ,warl****,2020.09.09. 23:46,4,8,https://news.naver.com/main/read.nhn?mode=LSD&...
